# 一些功能及用法演示

### 1.变量

In [1]:
from src.vqf_var_pool import VQFVarPool, VQFVar

[WARNING] ME(147207:140353854792512,MainProcess):2023-10-18-00:25:02.556.931 [mindspore/run_check/_check_version.py:102] MindSpore version 2.1.0 and cuda version 11.2.72 does not match, CUDA version [['10.1', '11.1', '11.6']] are supported by MindSpore officially. Please refer to the installation guide for version matching information: https://www.mindspore.cn/install.


#### 1.1.使用VQFVarPool生成p、q变量池

In [2]:
m, p, q = 56153, 241, 233
vp = VQFVarPool(m, [p, q])
vp.all_elements

['p0',
 'p1',
 'p2',
 'p3',
 'p4',
 'p5',
 'p6',
 'p7',
 'q0',
 'q1',
 'q2',
 'q3',
 'q4',
 'q5',
 'q6',
 'q7']

#### 1.2.如果p、q未给出真值则根据m二进制长度生成p、q变量池

默认p＞q

In [3]:
vp = VQFVarPool(m, None)
vp.all_elements

['p0',
 'p1',
 'p2',
 'p3',
 'p4',
 'p5',
 'p6',
 'p7',
 'p8',
 'p9',
 'p10',
 'p11',
 'p12',
 'p13',
 'p14',
 'p15',
 'q0',
 'q1',
 'q2',
 'q3',
 'q4',
 'q5',
 'q6',
 'q7']

#### 1.3.修改或新增变量

In [4]:
vp.update_var('z_1_2', 1, 'z_1_2')

输入参数意义见VQFVar

In [5]:
help(VQFVar)

Help on class VQFVar in module src.vqf_var_pool:

class VQFVar(builtins.object)
 |  VQFVar(name, expr_form, value)
 |  
 |  Args:
 |      name (str): Name of variable.
 |      expr_form (int): Expression form. Supported:
 |          {0: binary int, 1: x, 2: 1-x, 3: xy, 4: 1-xy}
 |      value (str): Variable name or a number.
 |  
 |  Methods defined here:
 |  
 |  __call__(self, obj)
 |      Get term.
 |  
 |  __init__(self, name, expr_form, value)
 |      Initialize a VQFVar object.
 |  
 |  clear(self, obj)
 |      Convert a non-variable item to a constant value.
 |  
 |  get_expr(self, obj)
 |      Get variable relation expression.
 |  
 |  get_var(self, obj)
 |      Get all independent variables.
 |  
 |  ----------------------------------------------------------------------
 |  Readonly properties defined here:
 |  
 |  is_const
 |      Check whether the variable is constant.
 |  
 |  is_ind_var
 |      Check whether the variable is independent variable.
 |  
 |  -----------------

#### 1.4.查看所有自变量

In [6]:
vp.all_variables

['p1',
 'p2',
 'p3',
 'p4',
 'p5',
 'p6',
 'p7',
 'p8',
 'p9',
 'p10',
 'p11',
 'p12',
 'p13',
 'p14',
 'p15',
 'q1',
 'q2',
 'q3',
 'q4',
 'q5',
 'q6',
 'q7',
 'z_1_2']

#### 1.5.建立给定变量与泡利算符关系

In [7]:
vp.build_var_oper_list(vp.all_elements[:4])
vp.all_elements[:4]

['p0', 'p1', 'p2', 'p3']

In [8]:
for v in vp.var_list:
    print(v)
    print(vp(v))

p1
1/2 [Z0] +
1/2 []
p2
1/2 [Z1] +
1/2 []
p3
1/2 [Z2] +
1/2 []


#### 1.6.理论上可以通过中间变量构造绝大部分关系式

需注意避免关系式闭合情况(无检错-死循环)

In [9]:
# p1 = p2*p3*p4
vp.update_var('m0', 3, ('p2', 'p3'))     # m0=p2*p3
vp.update_var('p1', 3, ('m0', 'p4'))     # p1=m0*p4
vp.get_expr('p1')   # 获取(表达式、包含变量)

('p2*p3*p4', ['p2', 'p3', 'p4'])

#### 1.7.常值项清理中间过程

In [10]:
# p2=0
# -> m0=0
# -> p1=0
vp.update_var('p2', 0, 0)
vp.clear()
[vp.get_expr(v) for v in ['p1', 'p2', 'p3', 'p4', 'm0']]

[('0', []), ('0', []), ('p3', ['p3']), ('p4', ['p4']), ('0', [])]

#### 1.8.常值与自变量判断

In [11]:
[(vp.v_dict[v].is_const, vp.v_dict[v].is_ind_var) for v in ['p2', 'p3']]

[(True, False), (False, True)]

### 2.子句

In [12]:
from src.vqf_clauses_lib import VQFClausesA

#### 2.1.生成子句集

In [13]:
m, p, q = 56153, 241, 233
vc = VQFClausesA(m, [p, q])

In [14]:
vc = VQFClausesA(m, None)
print(vc.all_variables)

['p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9', 'p10', 'p11', 'p12', 'p13', 'p14', 'p15', 'q1', 'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'z_1_2', 'z_2_3', 'z_2_4', 'z_3_4', 'z_3_5', 'z_4_5', 'z_4_6', 'z_5_6', 'z_5_7', 'z_5_8', 'z_6_7', 'z_6_8', 'z_6_9', 'z_7_8', 'z_7_9', 'z_7_10', 'z_8_9', 'z_8_10', 'z_8_11', 'z_9_10', 'z_9_11', 'z_9_12', 'z_10_11', 'z_10_12', 'z_10_13', 'z_11_12', 'z_11_13', 'z_11_14', 'z_12_13', 'z_12_14', 'z_12_15', 'z_13_14', 'z_13_15', 'z_13_16', 'z_14_15', 'z_14_16', 'z_14_17', 'z_15_16', 'z_15_17', 'z_15_18', 'z_16_17', 'z_16_18', 'z_16_19', 'z_17_18', 'z_17_19', 'z_17_20', 'z_18_19', 'z_18_20', 'z_18_21', 'z_19_20', 'z_19_21', 'z_20_21', 'z_20_22', 'z_21_22', 'z_21_23', 'z_22_23']


#### 2.2.子句形式统一

In [15]:
vc.n_clauses

24

子句集由$n_c$个子句组成: `list[list]`

In [16]:
vc.clauses[0:2]

[[{'p0', 'q0'}, (['p0', 'q0'], -1)],
 [{'p0', 'p1', 'q0', 'q1', 'z_1_2'},
  (['p0', 'q1'], ['p1', 'q0'], 0, [-2, 'z_1_2'])]]

每个子句由变量集和累加项组成: `list[set, tuple]`

In [17]:
vc.clauses[1]

[{'p0', 'p1', 'q0', 'q1', 'z_1_2'},
 (['p0', 'q1'], ['p1', 'q0'], 0, [-2, 'z_1_2'])]

变量集由表达该子句所需的变量名组成: `set[str]`

In [18]:
vc.clauses[1][0]

{'p0', 'p1', 'q0', 'q1', 'z_1_2'}

子句累加项由一系列数值、变量名、累乘项组成: `tuple[...]`

In [19]:
vc.clauses[1][1]

(['p0', 'q1'], ['p1', 'q0'], 0, [-2, 'z_1_2'])

#### 2.3.生成子句表达式

生成指定子句解析后的字符串表达式和表达式中包含的所有自变量

便于应用经典预处理操作

In [20]:
for i in range(3):
    print(vc.get_expr(i))

('(1)*(1)+(-1)', [])
('(1)*(q1)+(p1)*(1)+(0)+(-2)*(z_1_2)', ['z_1_2', 'p1', 'q1'])
('(1)*(q2)+(p1)*(q1)+(p2)*(1)+(z_1_2)+(0)+(-2)*(z_2_3)+(-4)*(z_2_4)', ['q2', 'z_2_4', 'q1', 'p2', 'z_1_2', 'p1', 'z_2_3'])


#### 2.4.哈密顿量

获取指定区间内子句的哈密顿量
$H_{c_lc_r}=\sum^{c_r}_{i=c_l} C^2_i$

$(c_l=0, c_r=n_c)$即为整个子句集的哈密顿量

In [21]:
c_l = 1
c_r = 3
ham, var_l = vc.get_ham(c_l, c_r)
ham

   9 [] +
-3/4 [Z0 Z2] +
-3/4 [Z1 Z2] +
-5/4 [Z3] +
 1/4 [Z0 Z1 Z3] +
 1/4 [Z1 Z3] +
 1/4 [Z0 Z3] +
 1/2 [Z3 Z4] +
 1/2 [Z2 Z3] +
  -1 [Z3 Z5] +
  -2 [Z3 Z6] +
-1/2 [Z1] +
-1/2 [Z0] +
-5/4 [Z4] +
-5/4 [Z2] +
 5/2 [Z5] +
   5 [Z6] +
 1/4 [Z0 Z1 Z4] +
 1/4 [Z0 Z1 Z2] +
-1/2 [Z0 Z1 Z5] +
  -1 [Z0 Z1 Z6] +
 1/4 [Z1 Z4] +
-1/2 [Z1 Z5] +
  -1 [Z1 Z6] +
 1/4 [Z0 Z4] +
-1/2 [Z0 Z5] +
  -1 [Z0 Z6] +
 1/2 [Z2 Z4] +
  -1 [Z4 Z5] +
  -2 [Z4 Z6] +
  -1 [Z2 Z5] +
  -2 [Z2 Z6] +
   4 [Z5 Z6]

泡利算符Z0~Zn(此处n=6)按顺序对应子句自变量列表

In [22]:
var_l

['q1', 'p1', 'z_1_2', 'q2', 'p2', 'z_2_3', 'z_2_4']

#### 2.5.生成子句集

与1.3同

In [ ]:
vc.update_var(name, expr_form, value)

#### 2.6.生成结果

所有变量赋值为1后获取结果

In [ ]:
_ = [vc.update_var(k, 0, 1) for k in vc.all_variables]
vc.get_result()

{'status': True, 'p': 255, 'q': 255, 'm': 56153, 'p_q_real': [241, 233]}

检查当前p、q代入到每个子句后是否满足值为0

In [ ]:
for i in range(vc.n_clauses):
    expr, v = vc.get_expr(i)
    if v:
        print(expr)
    else:
        print(eval(expr))

0
0
-2
-2
0
-6
-6
-4
-5
-6
-6
0
-1
-1
0
1


### 3.预处理

#### 3.1.